In [19]:
from bertopic import BERTopic
from datasets import load_dataset
import torch

In [26]:
pip install safetensors


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [2]:
dataset = load_dataset("mwarchalowski/grants", "no-shorts-no-dups")

In [3]:
from sentence_transformers import SentenceTransformer

In [4]:
embedding_model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/111k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [7]:
splits = dataset["train"].train_test_split(0.1)
splits

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'abstr', 'text_len'],
        num_rows: 114100
    })
    test: Dataset({
        features: ['id', 'title', 'abstr', 'text_len'],
        num_rows: 12678
    })
})

In [16]:
embeddings = embedding_model.encode(splits["train"]["abstr"], show_progress_bar=True)
embeddings

Batches:   0%|          | 0/3566 [00:00<?, ?it/s]

array([[-0.2721274 ,  0.39993554,  0.04004147, ...,  0.5322828 ,
        -0.15304367,  0.42033535],
       [ 0.37619624,  0.5639731 , -0.2674801 , ...,  0.09050045,
        -0.24711074,  0.07018428],
       [ 0.07103545,  0.40288356,  0.30178797, ...,  0.11804905,
        -0.3014268 ,  0.22825383],
       ...,
       [-0.13096468,  0.5218836 ,  0.28385654, ...,  0.2532104 ,
        -0.7755157 , -0.25065824],
       [ 0.03244508,  0.8145087 , -0.19566627, ..., -0.11640655,
        -0.32859585,  0.19593976],
       [ 0.39871967, -0.47737214,  1.6064315 , ...,  0.4581581 ,
        -0.25867087, -0.68136466]], dtype=float32)

In [17]:
embeddings.shape

(114100, 1024)

In [18]:
type(embeddings)

numpy.ndarray

In [21]:
torch_embeddings = torch.from_numpy(embeddings)

In [22]:
torch_embeddings.shape

torch.Size([114100, 1024])

In [25]:
torch.save(torch_embeddings, "embeddings.pt")

In [27]:
from safetensors.torch import save_file

In [29]:
tensors = {'embeddings': torch_embeddings}
save_file(tensors, "embeddings.safetensors")

In [49]:
from umap import UMAP
umap_model = UMAP(n_neighbors=10, n_components=8, min_dist=0.0, metric='cosine', random_state=42)
umap_model

UMAP(metric='cosine', min_dist=0.0, n_components=8, n_neighbors=10, random_state=42)

In [52]:
from hdbscan import HDBSCAN
hdbscan_model = HDBSCAN(min_cluster_size=50, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
hdbscan_model

HDBSCAN(min_cluster_size=50, prediction_data=True)

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))
vectorizer_model

CountVectorizer(min_df=2, ngram_range=(1, 2), stop_words='english')

In [53]:
topic_model = BERTopic(

  # Pipeline models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  # representation_model=representation_model,

  # Hyperparameters
  top_n_words=30,
  verbose=True
)


In [54]:
topics, probs = topic_model.fit_transform(splits["train"]["abstr"], embeddings)


2024-03-17 21:01:30,578 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-17 21:03:12,435 - BERTopic - Dimensionality - Completed ✓
2024-03-17 21:03:12,440 - BERTopic - Cluster - Start clustering the reduced embeddings
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been us

In [55]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,57440,-1_research_project_cancer_cells,"[research, project, cancer, cells, new, data, ...","[The prostate has rightly been called the ""mos..."
1,0,1557,0_alloys_materials_material_composite,"[alloys, materials, material, composite, mecha...",[Using combinatorial strategies to develop opt...
2,1,1505,1_software_security_br_br gt,"[software, security, br, br gt, lt br, gt, lt,...",[0237835 &lt;br/&gt;David Beazley&lt;br/&gt;Un...
3,2,1333,2_species_evolutionary_populations_evolution,"[species, evolutionary, populations, evolution...","[There are more than 1,8 million described spe..."
4,3,1297,3_students_lt br_lt_br gt,"[students, lt br, lt, br gt, br, gt, science, ...",[Undergraduate students will take part in a th...
...,...,...,...,...,...
249,248,53,248_social_social media_media_online,"[social, social media, media, online, disinfor...",[Recent events in volatile geo-political regio...
250,249,53,249_air pollution_pollution_air_exposure,"[air pollution, pollution, air, exposure, pm2,...",[Exposure to ambient air pollution particles i...
251,250,51,250_pah_exposure_coking_workers,"[pah, exposure, coking, workers, sampling, nit...",[The basic aims of the research are to: - harm...
252,251,51,251_swallowing_masseter_tongue_emg,"[swallowing, masseter, tongue, emg, chewing, m...",[The purpose of this study was to investigate ...


In [59]:
probs

array([0.       , 0.3562243, 1.       , ..., 1.       , 0.       ,
       1.       ])